Code to simulate TTL events in a roundabout way when playing back data in Open Ephys.  Basic premise is to feed events and start times into an arduino which will relay these as TTLs to the acquisition system. Now that I think about this it probably won't work since the sink is the FileReader and I would need the TTL in to work!!! But maybe?

In [20]:
import numpy
import pyfirmata
import atexit
from pickle import load
import numpy as np
import time

Import stimulation and recording time data!

In [6]:
stim_times_file = r'C:\Users\nkinsky\Documents\Working\Opto\Jackie671\Jackie_placestim_day2\Jackie_PRE_2020-10-07_10-48-13\stim_times_only.pkl'
continuous_times_file = r'C:\Users\nkinsky\Documents\Working\Opto\Jackie671\Jackie_placestim_day2\Jackie_PRE_2020-10-07_10-48-13\experiment1\recording1\continuous\Intan_Rec._Controller-100.0\timestamps.npy'

stim_times = load(open(stim_times_file, 'rb'))
cont_times = np.load(continuous_times_file)

In [19]:
stim_times['on_times'][0:10]

array([351.156 , 361.508 , 397.1232, 400.0976, 403.1648, 406.2168,
       410.4192, 421.0904, 426.0248, 430.6792])

Define some useful functions here for handling the arduino

In [24]:
def initialize_arduino(
    port="COM3",
    record_pin=7,
    LED_pin=2):
    """20210202: No try/except for now because I want to see an error when setting things up for now!
    Not sure shock_io_pin is entirely necessary - just send shock_box_pin to both shock box and open ephys"""
    
    print('Initializing arduino. Thank you for your patience!')
    # try:
    board = pyfirmata.Arduino(port)
  
    pins = {'record_pin' : record_pin, 'LED_pin' : LED_pin} 

    # initialize cleanup function
    atexit.register(shutdown_arduino, board)
    
    return board, pins

def shutdown_arduino(board):
    """cleanup function to shutdown arduino in case of suddent exit"""
    if isinstance(board, pyfirmata.Arduino):
        print("Shutting down arduino")
        board.exit()

def pin_on(board, pin):
    board.digital[pin].write(1)  # signal to shock box

def pin_off(board, pin):
    board.digital[pin].write(0)

Load in `Jackie_placestim2` session to openephys GUI by selecting the `structure.oebin` file for the "PRE" session.  Then select option to remote trigger

In [34]:
shutdown_arduino(board)
on = [0.5, 0.5, 0.5, 0.5]
off = [0.5, 0.5, 0.5, 0.5]
board, pins = initialize_arduino()
pin_off(board, pins['LED_pin'])
for a, b in zip(on, off):
    pin_on(board, pins['LED_pin'])
    time.sleep(a)
    pin_off(board, pins['LED_pin'])
    time.sleep(b)
shutdown_arduino(board)

Shutting down arduino
Initializing arduino. Thank you for your patience!
Shutting down arduino


Now run the following code to try to start the recording and then stop it 10 seconds later.

In [36]:
shutdown_arduino(board)
board, pins = initialize_arduino()
for sec in [5, 4, 3, 2, 1]:
    print('Starting in t minus ' + str(sec) + ' seconds')
    time.sleep(1)
pin_on(board, pins['record_pin'])
for sec in [7, 6, 5, 4, 3, 2, 1]:
    print('Stopping in t minus ' + str(sec) + ' seconds')
    time.sleep(1)
pin_off(board, pins['record_pin'])

Shutting down arduino
Initializing arduino. Thank you for your patience!
Starting in t minus 5 seconds
Starting in t minus 4 seconds
Starting in t minus 3 seconds
Starting in t minus 2 seconds
Starting in t minus 1 seconds
Stopping in t minus 7 seconds
Stopping in t minus 6 seconds
Stopping in t minus 5 seconds
Stopping in t minus 4 seconds
Stopping in t minus 3 seconds
Stopping in t minus 2 seconds
Stopping in t minus 1 seconds


Ok, this works but ONLY if I have already hit play. So I'll need to synchronize off of something else, actually.  Geesh.

When above is verified, try starting at 5 minutes using the following code:

In [ ]:
OE_start_time = 5*60  # seconds